In [88]:

import numpy as np
import scipy as sp
from scipy.special import gammaln
import scipy.misc

def index_sample(p):
    '''
    Desc: Samples from n topics distributed multinomially and returns topic number
    input: p - A one dimensional array of float64 type that contains the probability for each topic
    output: an Integer specifying which topic was chosen from a multinomial distribution
    '''
    return np.random.multinomial(1,p).argmax()

def word_indices(vec):
    '''
    Desc: Take a vector of word counts from a document and create a generator for word indices
    input: A vector from a Document Term Frequency matrix for one document.
    output: A generator object to store the word indices when called
    '''
    for idx in vec.nonzero()[0]:
        for i in xrange(int(vec[idx])):
            yield idx

def log_multi_beta(alpha, K = None):
    """
    Desc: Compute the logarithm of the multinomial beta function
    input: alpha - A vector with type float64 or a scaler of float64
           K - An integer that, if alpha is a scalar, multiplies the log by K
    output: a float64 with value of the logarithm of the multinomial beta
    """
    
    if K is None:
        return np.sum(gammaln(alpha) - gammaln(np.sum(alpha)))
    else:
        return K * gammaln(alpha) - gammaln(K * alpha)
    
class LdaSampler(object):
    
    def __init__(self, ntopics, alpha = .1, beta = .1):
        """
        Desc: Initialize values for our class object
        alpha: a float scalar
        beta: a float scalar
        ntopics: an integer for the number of topics
        """
        if not isinstance(alpha, float):
            raise Exception(" Initial value for alpha must be a floating point number (.3)")
        
        if not isinstance(beta, float):
            raise Exception(" Initial value for beta must be a floating point number (.3)")
            
        if not isinstance(ntopics, int):
            raise Exception(" The number of topics must be an integer")
            
        
        self.ntopics = ntopics
        self.alpha = alpha
        self.beta = beta
    
    def _initialize(self,matrix):
        '''
        Initialize:
        NZM: size(#Docs X #Topics) numpy array with type float 64
            The number of times document M and topic Z interact
        
        NZW: size(#Topics X #Words) numpy array with type float64
            The number of times topic Z and word W interact
        
        NM:  size(#Docs) numpy array with type float64
            Sum of documents occurances by topic and word
        
        NZ:  size(#Topics) numpy array with type float64
            Sum of Topic occurences by word and document
        
        Topics: size(?) An empty set
           Will come back to this
        '''
        ndocs, vsize = matrix.shape
        
        self.NZM = np.zeros((ndocs, self.ntopics))
        self.NZW = np.zeros((self.ntopics, vsize))
        self.NM  = np.zeros(ndocs)
        self.NZ  = np.zeros(self.ntopics)
        self.topics = {}
        
        for m in xrange(ndocs):
            # Iterates over i, doc_length - 1, and w, the size of unique_words - 1
            for i, w, in enumerate(word_indices(matrix[m,:])):
                # Initialize a random topic for each word
                z = np.random.randint(self.ntopics)
                self.NZM[m,z] += 1
                # Why is NM being +1'd for each i,w?
                self.NM[m] += 1
                self.NZW[z,w] += 1
                self.NZ[z] += 1
                self.topics[(m,i)] = z
    
    def _conditional_distribution(self, m, w):
        '''
        Desc: Compute the conditional distribution of words in document and topic
        Input: m: An integer representing the column index of the document
               w: The generator object from word_indices
        
        Output: p_z: An array size(w X 1) containing probabilities for topics of word
        '''
        vsize = self.NZW.shape[1]
        left = (self.NZW[:,w] + self.beta) / (self.NZ + self.beta * vsize)
        right = (self.NZM[m,:] + self.alpha) / (self.NM[m] + self.alpha * self.ntopics)
        p_z = abs(left * right)
        p_z /= np.sum(p_z)
        return p_z
    
    def loglikelihood(self):
        '''
        Desc: Compute the log likelihood that the model generated the data
        Input: self references
        Output: lik: float of the log likelihood
        '''
        # Why are these being repeated here?
        vsize = self.NZW.shape[1]
        ndocs = self.NZM.shape[0]
        lik = 0
        
        for z in xrange(self.ntopics):
            lik += log_multi_beta(self.NZW[z,:] + self.beta)
            lik -= log_multi_beta(self.beta, vsize)
        
        for m in xrange(ndocs):
            lik += log_multi_beta(self.NZM[m,:] + self.alpha)
            lik -= log_multi_beta(self.alpha, self.ntopics)
        
        return lik
    
    def phi(self):
        '''
        Desc: Compute the probability of a word given topic
        Input: Self references
        ???
        ?Output: An array of size(1 X Words) containing the probabilities for a given topic
        ???
        '''
        num = self.NZW + self.beta
        num /= np.sum(num, axis = 1)[:, np.newaxis]
        return num
    
    def run(self, matrix, maxiter = 30, burnin = 10):
        '''
        Desc: Perform Gibbs sampling for maxiter iterations
        
        Input: matrix - An array that is a Document Term Frequency Matrix
               maxiter - An integer with the number of iterations
               Burnin - TBA: An integer of the number of burnins
               
        Output: phi() the probability of a word given a document
        '''
        
        n_docs, vsize = matrix.shape
        self._initialize(matrix)
        
        for iteration in xrange(maxiter):
            for burn in xrange(burnin):
                for m in xrange(n_docs):
                    for i,w in enumerate(word_indices(matrix[m,:])):
                        z = self.topics[(m,i)]
                    
                        # Why minus one?
                        self.NZM[m,z] -= 1
                        self.NM[m] -= 1
                        self.NZW[z,w] -= 1
                        self.NZ[z] -= 1
                    
                        p_z = self._conditional_distribution(m,w)
                        z = index_sample(p_z)
                    
                        self.NZM[m,z] += 1
                        self.NM[m] += 1
                        self.NZW[z,w] += 1
                        self.NZ[z] += 1
                        
                yield self.phi()

                
                  
                    

In [73]:

import os
import shutil

N_TOPICS = 10
DOCUMENT_LENGTH = 100
FOLDER = "topicimg"

def vertical_topic(width, topic_index, document_length):
    """
    Generate a topic whose words form a vertical bar.
    """
    m = np.zeros((width, width))
    m[:, topic_index] = int(document_length / width)
    return m.flatten()

def horizontal_topic(width, topic_index, document_length):
    """
    Generate a topic whose words form a horizontal bar.
    """
    m = np.zeros((width, width))
    m[topic_index, :] = int(document_length / width)
    return m.flatten()

def save_document_image(filename, doc, zoom=2):
    """
    Save document as an image.
    doc must be a square matrix
    """
    height, width = doc.shape
    zoom = np.ones((width*zoom, width*zoom))
    # imsave scales pixels between 0 and 255 automatically
    sp.misc.imsave(filename, np.kron(doc, zoom))

def gen_word_distribution(ntopics, document_length):
    """
    Generate a word distribution for each of the ntopics.
    """
    width = ntopics / 2
    vsize = width ** 2
    m = np.zeros((ntopics, vsize))

    for k in range(width):
        m[k,:] = vertical_topic(width, k, document_length)

    for k in range(width):
        m[k+width,:] = horizontal_topic(width, k, document_length)

    m /= m.sum(axis=1)[:, np.newaxis] # turn counts into probabilities

    return m

def gen_document(word_dist, ntopics, vsize, length=DOCUMENT_LENGTH, alpha=0.1):
    """
    Generate a document:
    1) Sample topic proportions from the Dirichlet distribution.
    2) Sample a topic index from the Multinomial with the topic
       proportions from 1).
    3) Sample a word from the Multinomial corresponding to the topic
       index from 2).
    4) Go to 2) if need another word.
    """
    theta = np.random.mtrand.dirichlet([alpha] * ntopics)
    v = np.zeros(vsize)
    for n in range(length):
        z = index_sample(theta)
        w = index_sample(word_dist[z,:])
        v[w] += 1
    return v

def gen_documents(word_dist, ntopics, vsize, n=500):
    """
    Generate a document-term matrix.
    """
    m = np.zeros((n, vsize))
    for i in xrange(n):
        m[i, :] = gen_document(word_dist, ntopics, vsize)
    
    return m

if os.path.exists(FOLDER):
    shutil.rmtree(FOLDER)
os.mkdir(FOLDER)



In [ ]:
width = N_TOPICS / 2
vsize = width ** 2
word_dist = gen_word_distribution(N_TOPICS, DOCUMENT_LENGTH)
matrix = gen_documents(word_dist, N_TOPICS, vsize)
sampler = LdaSampler(N_TOPICS)

for it, phi in enumerate(sampler.run(matrix)):
    print "Iteration", it
    print "Likelihood", sampler.loglikelihood()

    if it % 5 == 0:
        for z in range(N_TOPICS):
            save_document_image("topicimg/topic%d-%d.png" % (it,z),
            phi[z,:].reshape(width,-1))



Iteration 0
Likelihood -10940613.7489
Iteration 1
Likelihood -10893431.8518
Iteration 2
Likelihood -10811639.7223
Iteration 3
Likelihood -10735797.9854
Iteration 4
Likelihood -10666011.6539
Iteration 5
Likelihood -10591724.4034
Iteration 6
Likelihood -10518229.2302
Iteration 7
Likelihood -10444880.6194
Iteration 8
Likelihood -10372297.044
Iteration 9
Likelihood -10301053.0435
Iteration 10
Likelihood -10230344.1454
Iteration 11
Likelihood -10158971.6791
Iteration 12
Likelihood -10086640.5294
Iteration 13
Likelihood -10012641.206
Iteration 14
Likelihood -9941714.0673
Iteration 15
Likelihood -9868821.70907
Iteration 16
Likelihood -9797512.35174
Iteration 17
Likelihood -9724779.86518
Iteration 18
Likelihood -9654301.46867
Iteration 19
Likelihood -9581852.81952
Iteration 20
Likelihood -9510058.9357
Iteration 21
Likelihood -9439599.46918
Iteration 22
Likelihood -9367880.27203
Iteration 23
Likelihood -9296062.68742
Iteration 24
Likelihood -9223015.61027
Iteration 25
Likelihood -9148658.93831


In [69]:
for it, phi in enumerate(sampler.run(matrix)):
    print "Iteration", it
    print "Likelihood", sampler.loglikelihood()

    if it % 5 == 0:
        for z in range(N_TOPICS):
            save_document_image("topicimg/topic%d-%d.png" % (it,z),
            phi[z,:].reshape(width,-1))